# Cracking sources of instability

Cracking is a possible instability source of compact objects when these are subjected to local density perturbations. The stability criteria is based on the sign change of the following quantities

$$ \delta \mathcal{R}_{p}=\frac{P^{\prime \prime}}{\rho^{\prime}} \delta \rho=\left[\left(v^{2}\right)^{\prime}+v^{2} \frac{\rho^{\prime \prime}}{\rho^{\prime}}\right] \delta \rho, $$

$$ \delta \mathcal{R}_{g}=\left[\frac{\partial \mathcal{R}_{g}}{\partial \rho}+\frac{\partial \mathcal{R}_{g}}{\partial P} v^{2}+\frac{\partial \mathcal{R}_{g}}{\partial m} \frac{4 \pi r^{2} \rho}{\rho^{\prime}}\right] \delta \rho,  $$

$$ \delta \mathcal{R}_{a}=2\left[\frac{v^{2}-v_{\perp}^{2}}{r}\right] \delta \rho $$
where 

$$ \frac{\partial \mathcal{R}_{g}}{\partial \rho}=\frac{m+4 \pi r^{3} P}{r(r-2 m)}, \frac{\partial \mathcal{R}_{g}}{\partial P}=\frac{m+4 \pi r^{3}(\rho+2 P)}{r(r-2 m)}, \frac{\partial \mathcal{R}_{g}}{\partial m}=\frac{(\rho+P)\left(1+8 \pi r^{2} P\right)}{(r-2 m)^{2}}   $$

and

$$ v^{2} = \frac{dP}{d\rho} \quad , \quad v_{\perp}^{2} = \frac{dP_{\perp}}{d\rho} $$

## Isotropic case

In this case $P=P_{\perp}$ so that $v=v_{\perp}$ and $\delta \mathcal{R}_{a}=0$. The possible change of sign is due to $\mathcal{R}_{p}$ and $\mathcal{R}_{g}$.

### Calculating $\mathcal{R}_{p}$

In [2]:
import os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker
import matplotlib as mpl
from scipy import integrate, interpolate
from Physical_Const import *
plt.rc('text', usetex=True)
plt.rc('font', family='serif')
plt.rc('axes', labelsize=21)
plt.rc('xtick', labelsize=14)
plt.rc('ytick', labelsize=14)
plt.rc('legend',fontsize= 14,markerscale=1.5)
plt.rcParams['axes.spines.right'] = True
plt.rcParams['axes.spines.top'] = True
mpl.rcParams['axes.linewidth'] = 1.0